importing 

In [114]:
import pandas as pd
import numpy as np
import time
import os
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [115]:
df = pd.read_csv(r"C:\Users\P PAVAN REDDY\Documents\My Files\Projects\Recommendation System - Movies\movies.csv")

In [116]:
movies = df.copy()

In [117]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6147 entries, 0 to 6146
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   title                         6147 non-null   object
 1   release_year                  6147 non-null   int64 
 2   imdb_id                       6147 non-null   object
 3   poster                        6147 non-null   object
 4   tmdb_id                       6147 non-null   int64 
 5   cast                          6147 non-null   object
 6   director                      6147 non-null   object
 7   genres                        6147 non-null   object
 8   production_companies          6147 non-null   object
 9   cast_display                  6147 non-null   object
 10  director_display              6147 non-null   object
 11  genres_display                6147 non-null   object
 12  production_companies_display  6147 non-null   object
 13  title_search      

In [118]:
movies.columns

Index(['title', 'release_year', 'imdb_id', 'poster', 'tmdb_id', 'cast',
       'director', 'genres', 'production_companies', 'cast_display',
       'director_display', 'genres_display', 'production_companies_display',
       'title_search', 'document'],
      dtype='object')

creating tags column by concatinating 'cast', 'director', 'genres', 'production_companies'

In [133]:
movies['tags'] = (movies['cast'] + " " + movies['director'] + " " 
                       + movies['genres'] + movies['production_companies']
                      )

In [134]:
movies.head()

,title,release_year,imdb_id,poster,tmdb_id,cast,director,genres,production_companies,cast_display,director_display,genres_display,production_companies_display,title_search,document,tags
0,29th Street,1991,tt0101252,https://m.media-amazon.com/images/M/MV5BMWY2MT...,43267,AnthonyLaPaglia DannyAiello LainieKazan FrankP...,GeorgeGallo,Comedy,20thCenturyFox,"Anthony La Paglia, Danny Aiello, Lainie Kaza...",George Gallo,Comedy,20th Century Fox,29thstreet,AnthonyLaPaglia DannyAiello LainieKazan FrankP...,AnthonyLaPaglia DannyAiello LainieKazan FrankP...
1,The Addams Family,1991,tt0101272,https://m.media-amazon.com/images/M/MV5BODc1Nm...,2907,RaúlJuliá AnjelicaHuston ChristopherLloyd Chri...,BarrySonnenfeld,Comedy Fantasy,ScottRudinProductions OrionPictures Paramount,"Raúl Juliá, Anjelica Huston, Christopher Llo...",Barry Sonnenfeld,"Comedy, Fantasy","Scott Rudin Productions, Orion Pictures, Par...",theaddamsfamily,RaúlJuliá AnjelicaHuston ChristopherLloyd Chri...,RaúlJuliá AnjelicaHuston ChristopherLloyd Chri...
2,All I Want for Christmas,1991,tt0101301,https://m.media-amazon.com/images/M/MV5BMTk3Mj...,13644,HarleyJaneKozak JameySheridan ThoraBirch Ethan...,RobLieberman,Family Drama Comedy,Paramount,"Harley Jane Kozak, Jamey Sheridan, Thora Bir...",Rob Lieberman,"Family, Drama, Comedy",Paramount,alliwantforchristmas,HarleyJaneKozak JameySheridan ThoraBirch Ethan...,HarleyJaneKozak JameySheridan ThoraBirch Ethan...
3,American Kickboxer,1991,tt0101325,https://m.media-amazon.com/images/M/MV5BM2E2Nm...,110830,JohnBarrett KeithVitali TerryNorton BradMorris...,FransNel,Action,GlobalPictures DistantHorizon CannonGroup,"John Barrett, Keith Vitali, Terry Norton, B...",Frans Nel,Action,"Global Pictures, Distant Horizon, Cannon Group",americankickboxer,JohnBarrett KeithVitali TerryNorton BradMorris...,JohnBarrett KeithVitali TerryNorton BradMorris...
4,An American Tail: Fievel Goes West,1991,tt0101329,https://m.media-amazon.com/images/M/MV5BZmUyOT...,10380,PhillipGlasser JamesStewart EricaYohn CathyCav...,SimonWells,Adventure Animation Family Western,AmblinEntertainment UniversalPictures Amblimation,"Phillip Glasser, James Stewart, Erica Yohn, ...",Simon Wells,"Adventure, Animation, Family, Western","Amblin Entertainment, Universal Pictures, Am...",anamericantailfievelgoeswest,PhillipGlasser JamesStewart EricaYohn CathyCav...,PhillipGlasser JamesStewart EricaYohn CathyCav...


* TfidfVectorizer

In [135]:
movies[movies['title'] == 'Iron Man']

,title,release_year,imdb_id,poster,tmdb_id,cast,director,genres,production_companies,cast_display,director_display,genres_display,production_companies_display,title_search,document,tags
3254,Iron Man,2008,tt0371746,https://m.media-amazon.com/images/M/MV5BMTczNT...,1726,RobertDowneyJr TerrenceHoward JeffBridges Gwyn...,JonFavreau,Action ScienceFiction Adventure,MarvelStudios,"Robert Downey Jr, Terrence Howard, Jeff Brid...",Jon Favreau,"Action, Science Fiction, Adventure",Marvel Studios,ironman,RobertDowneyJr TerrenceHoward JeffBridges Gwyn...,RobertDowneyJr TerrenceHoward JeffBridges Gwyn...


In [136]:
tfidf_tags = TfidfVectorizer()   #for tags
tfidf_tags

TfidfVectorizer()

In [137]:
tfidf_tags_matrix = tfidf_tags.fit_transform(movies['tags'])  #tags -> tfidf vector
tfidf_tags_matrix

<6147x24390 sparse matrix of type '<class 'numpy.float64'>'
	with 73040 stored elements in Compressed Sparse Row format>

* cosine similariry

In [138]:
tags_similarity_array = cosine_similarity(tfidf_tags_matrix)
tags_similarity_array

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.72649311,
        0.0135681 ],
       [0.        , 0.        , 0.        , ..., 0.72649311, 1.        ,
        0.01421929],
       [0.        , 0.        , 0.        , ..., 0.0135681 , 0.01421929,
        1.        ]])

In [139]:
#pickle.dump(tags_similarity_array, open('similarity_array.pkl', 'wb'))

* building function

In [140]:
def recommend(t):
    
    if t not in movies['title'].values:
        print("The movie title you entered is not in our database. Please try entering a different title.")
        
    else:
    
        if len(movies.loc[movies['title']==t]) > 1:
        
            #printing release year of movies with same name
            print(movies.loc[movies['title']==t]['release_year'].values)
            print('please provide release year...from above')
        
            r_year = int(input('Enter the release year :'))
        
            if r_year not in movies.loc[movies['title']==t]['release_year'].values:
                print("\n The release year {} you entered is not correct. We have taken the movie that was released first.".format(r_year))
                
                idx =  movies.loc[movies['title']== t].index[0]
                
            else:
                idx =  movies.loc[(movies['title'] == t) & (movies['release_year'] == r_year)].index[0]
    
    
        else:
            idx =  movies.loc[movies['title']== t].index[0]
        
        
        score_lst = list(enumerate(tags_similarity_array[idx]))   #give index and score
        score_lst = sorted(score_lst, key = lambda x:x[1] ,reverse=True)     #sorting descending
        score_lst = score_lst[1:17] #excluding first item since it's the requested movie itself and giving top10
    
        rcmd_lst = []                                             
        for i in range(len(score_lst)):
            a = score_lst[i][0]
            rcmd_lst.append(movies.iloc[a]['title']) 
        
        
        print('\n Here are the recommendations for the movie: {}'.format(t))
        
        return rcmd_lst

In [141]:
recommend('The Avengers')

[1998 2012]
please provide release year...from above


ValueError: invalid literal for int() with base 10: ''

In [12]:
movies.loc[(movies['title'] =='The Avengers') & (movies['release_year'] == 2012) ].index[0]

4035

In [13]:
movies['poster'][0]

'https://m.media-amazon.com/images/M/MV5BMWY2MTY1NWUtN2E1NC00MTg2LWFkOWQtNmE0ODJlMDZhYzUyXkEyXkFqcGdeQXVyNjMwMjk0MTQ@._V1_.jpg'

building engine with different weitages for different features

* genres

In [14]:
#tfidf
genres_tfidf = TfidfVectorizer()
genres_tfidf

TfidfVectorizer()

In [15]:
tfidf_genres_matrix = genres_tfidf.fit_transform(movies['genres'])  #tags -> tfidf vector
tfidf_genres_matrix

<6141x23 sparse matrix of type '<class 'numpy.float64'>'
	with 15445 stored elements in Compressed Sparse Row format>

In [16]:
genres_similarity_array = cosine_similarity(tfidf_genres_matrix)
genres_similarity_array

array([[1.        , 0.49176832, 0.4693157 , ..., 0.        , 0.33940566,
        0.34926628],
       [0.49176832, 1.        , 0.23079459, ..., 0.        , 0.16690895,
        0.17175809],
       [0.4693157 , 0.23079459, 1.        , ..., 0.13670472, 0.58723263,
        0.16391615],
       ...,
       [0.        , 0.        , 0.13670472, ..., 1.        , 0.        ,
        0.57973215],
       [0.33940566, 0.16690895, 0.58723263, ..., 0.        , 1.        ,
        0.11854295],
       [0.34926628, 0.17175809, 0.16391615, ..., 0.57973215, 0.11854295,
        1.        ]])

* other tags

In [17]:
movies['other_tags'] = movies['cast'] + " " + movies['director'] + " " + movies['production_companies']

In [18]:
tfidf_othertags_matrix = tfidf_tags.fit_transform(movies['other_tags'])  #tags -> tfidf vector
tfidf_othertags_matrix

<6141x21686 sparse matrix of type '<class 'numpy.float64'>'
	with 63656 stored elements in Compressed Sparse Row format>

In [19]:
othertags_similarity_array = cosine_similarity(tfidf_othertags_matrix)
othertags_similarity_array

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.03396061, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.03396061, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

* building function

In [20]:
def weighted_recommend(t):
    
    if t not in movies['title'].values:
        print("The movie title you entered is not in our database. Please try entering a different title.")
        
    else:
    
        if len(movies.loc[movies['title']==t]) > 1:
        
            #printing release year of movies with same name
            print(movies.loc[movies['title']==t]['release_year'].values)
            print('please provide release year...from above')
        
            r_year = int(input('Enter the release year :'))
            
            if r_year not in movies.loc[movies['title']==t]['release_year'].values:
                print("Please try to select a valid release year for the movie from above")
                
            else:
                idx =  movies.loc[(movies['title'] == t) & (movies['release_year'] == r_year)].index[0]
    
        else:
            idx =  movies.loc[movies['title']== t].index[0]
        
        
        score_lst = (genres_similarity_array[idx] * 0.4 + othertags_similarity_array[idx] * 0.6)
        score_lst = list(enumerate(score_lst))   #give index and score
        score_lst = sorted(score_lst, key = lambda x:x[1] ,reverse=True)     #sorting descending
        score_lst = score_lst[1:15] #excluding first item since it's the requested movie itself and giving top10
    
        rcmd_lst = []                                             
        for i in range(len(score_lst)):
            a = score_lst[i][0]
            rcmd_lst.append(movies['title'][a]) 
        
        
        print('\n Here are the recommendations for the movie: {}'.format(t))
        
        return rcmd_lst

In [21]:
weighted_recommend('I, Robot')


 Here are the recommendations for the movie: I, Robot


['Battle: Los Angeles',
 'Terminator: Dark Fate',
 'After Earth',
 'Cyborg Cop',
 'Monolith',
 'Cyborg Cop II',
 'Black Scorpion',
 'Barb Wire',
 'Space Marines',
 'Toad Warrior',
 'Double Team',
 'Soldier',
 'The Matrix',
 'Universal Soldier: The Return']

In [22]:
recommend('I, Robot')


 Here are the recommendations for the movie: I, Robot


['Welcome to Mooseport',
 'Bright',
 'Seven Pounds',
 'Hitch',
 'After Earth',
 'The Pursuit of Happyness',
 'Bad Boys for Life',
 'Collateral Beauty',
 'Ali',
 '28 Days',
 'King Richard',
 'Hancock',
 'I Am Legend',
 'This Means War']

In [23]:
def recommend(t):
    
    if t not in movies['title'].values:
        print("The movie title you entered is not in our database. Please try entering a different title.")
        
    else:
    
        if len(movies.loc[movies['title']==t]) > 1:
        
            #printing release year of movies with same name
            print(movies.loc[movies['title']==t]['release_year'].values)
            print('please provide release year...from above')
        
            r_year = int(input('Enter the release year :'))
        
            if r_year not in movies.loc[movies['title']==t]['release_year'].values:
                print("\n The release year {} you entered is not correct. We have taken the movie that was released first.".format(r_year))
                
                idx =  movies.loc[movies['title']== t].index[0]
                
            else:
                idx =  movies.loc[(movies['title'] == t) & (movies['release_year'] == r_year)].index[0]
    
    
        else:
            idx =  movies.loc[movies['title']== t].index[0]
        
        
        score_lst = list(enumerate(tags_similarity_array[idx]))   #give index and score
        score_lst = sorted(score_lst, key = lambda x:x[1] ,reverse=True)     #sorting descending
        score_lst = score_lst[1:15] #excluding first item since it's the requested movie itself and giving top10
    
        rcmd_lst = []                                             
        for i in range(len(score_lst)):
            a = score_lst[i][0]
            rcmd_lst.append(movies['title'][a]) 
        
        
        print('\n Here are the recommendations for the movie: {}'.format(t))
        
        return rcmd_lst

In [24]:
recommend("The Avengers")

[1998 2012]
please provide release year...from above


Enter the release year : 2012



 Here are the recommendations for the movie: The Avengers


['Avengers: Age of Ultron',
 'Avengers: Endgame',
 'Avengers: Infinity War',
 'Captain America: Civil War',
 'Iron Man',
 'Captain America: The Winter Soldier',
 'Ant-Man',
 'Iron Man 2',
 'Guardians of the Galaxy',
 'Serenity',
 'The Incredible Hulk',
 'Zodiac',
 'Eternals',
 'The Perfect Score']

recommendation engine for user's profile

In [162]:
movies=pd.read_csv(r'C:\Users\P PAVAN REDDY\Documents\My Files\Projects\Recommendation System - Movies\movies_data.csv')

In [163]:
cast = input("enter your favourite actors: ")
director = input("enter your favourite director: ")
genres = input("enter your favourite genres: ")

In [164]:
cast, director, genres

('', 'h', '1')

In [165]:
cast = cast.replace(' ', '')
cast

''

In [166]:
director = director.replace(' ', '')
director

'h'

In [167]:
director = director.replace(' ', '')
director

'h'

In [168]:
genres

'1'

In [169]:
document = cast + " " + director + " " + genres + " " + 'JonFavreau'
document

' h 1 JonFavreau'

In [170]:
pd.DataFrame([document], columns=['document'])

,document
0,h 1 JonFavreau


In [171]:
user_input_vector = tfidf_tags.transform([document])


In [172]:
# user_input_vector = tfidf_tags.transform(pd.DataFrame([document], columns=['document']))
# user_input_vector.toarray()[0]

In [173]:
user_input_similarity_matrix = cosine_similarity(tfidf_tags_matrix, user_input_vector)
user_input_similarity_matrix

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [174]:
user_input_similarity_matrix[:,0]

array([0., 0., 0., ..., 0., 0., 0.])

In [177]:
movies['user_recommendation_score'] = user_input_similarity_matrix[:,0]

ValueError: Length of values (6147) does not match length of index (9520)

In [176]:
movies.sort_values(by=['user_recommendation_score'], ascending=False)[:20]

KeyError: 'user_recommendation_score'

In [103]:
df_sorted = df.sort_values(by='user_similarity', ascending=False)

(6147,)

In [101]:
user_input_similarity_matrix.shape

(1, 6147)

In [72]:
movies['cast'].nunique()

6141

In [178]:
cast_tfidf_object = TfidfVectorizer()
cast_tfidf_object

TfidfVectorizer()

In [179]:
cast_tfidf_vector = cast_tfidf_object.fit_transform(movies['cast'])
cast_tfidf_vector

<9520x17334 sparse matrix of type '<class 'numpy.float64'>'
	with 54872 stored elements in Compressed Sparse Row format>

In [180]:
director_tfidf_object = TfidfVectorizer()
director_tfidf_vector = director_tfidf_object.fit_transform(movies['director'])
director_tfidf_vector

<9520x4601 sparse matrix of type '<class 'numpy.float64'>'
	with 9602 stored elements in Compressed Sparse Row format>

In [181]:
genres_tfidf_object = TfidfVectorizer()
genres_tfidf_vector = genres_tfidf_object.fit_transform(movies['genres'])
genres_tfidf_vector

<9520x66 sparse matrix of type '<class 'numpy.float64'>'
	with 22221 stored elements in Compressed Sparse Row format>

In [241]:
def profile_recommendation(cast='', director='', genres=''):

        cast = cast.replace(" ", "").replace(',', ' ').lower()
        director = director.replace(" ", "").replace(',', ' ').lower()
        genres = genres.replace(" ", "").replace(',', ' ').lower()
                                 
        
        cast_vector, director_vector, genres_vector = (cast_tfidf_object.transform([cast]), director_tfidf_object.transform([director]), 
                                                       genres_tfidf_object.transform([genres]))


        cast_similarities, director_similarities, genres_similarities = (cosine_similarity(cast_tfidf_vector, cast_vector), 
                                                                        cosine_similarity(director_tfidf_vector, director_vector), 
                                                                        cosine_similarity(genres_tfidf_vector, genres_vector))

        user_input_similarity_scores = (cast_similarities * 4) + (director_similarities) + (genres_similarities * 0.5)

        movies['user_recommendation_score'] = user_input_similarity_scores[:,0]

        recommendations = movies.sort_values(by=['user_recommendation_score'], ascending=False)#[:30]
        
        
        return recommendations['title'].head(10)#.to_dict(orient='records')

profile_recommendation(cast='Robert Downey Jr, johnny depp, allu arjun', director='james cameron', genres='ScienceFiction')

179     Terminator 2: Judgment Day
6143      Avatar: The Way of Water
3622                        Avatar
3680                    Iron Man 2
5134        Avengers: Infinity War
4617       Avengers: Age of Ultron
4035                  The Avengers
5334             Avengers: Endgame
8953           Duvvada Jagannadham
4967        Spider-Man: Homecoming
Name: title, dtype: object

3680                    Iron Man 2
1745          The Astronaut's Wife
5134        Avengers: Infinity War
4617       Avengers: Age of Ultron
4035                  The Avengers
5334             Avengers: Endgame
4775    Captain America: Civil War
6142                    Iron Man 3
3254                      Iron Man
4967        Spider-Man: Homecoming
Name: title, dtype: object

2

In [192]:
movies['tags'] = (movies['cast'] + " " + movies['director'] + " " 
                       + movies['genres']
                      )

In [193]:
tfidf_tags = TfidfVectorizer()   #for tags
tfidf_tags

TfidfVectorizer()

In [194]:
tfidf_tags_matrix = tfidf_tags.fit_transform(movies['tags'])  #tags -> tfidf vector
tfidf_tags_matrix

<9520x21459 sparse matrix of type '<class 'numpy.float64'>'
	with 86160 stored elements in Compressed Sparse Row format>

In [195]:
tags_similarity_array = cosine_similarity(tfidf_tags_matrix)
tags_similarity_array

array([[1.        , 0.00847728, 0.00824717, ..., 0.        , 0.00830682,
        0.00878918],
       [0.00847728, 1.        , 0.00841422, ..., 0.02824543, 0.00847507,
        0.0089672 ],
       [0.00824717, 0.00841422, 1.        , ..., 0.00611204, 0.00824502,
        0.01535967],
       ...,
       [0.        , 0.02824543, 0.00611204, ..., 1.        , 0.03358161,
        0.00651372],
       [0.00830682, 0.00847507, 0.00824502, ..., 0.03358161, 1.        ,
        0.00878688],
       [0.00878918, 0.0089672 , 0.01535967, ..., 0.00651372, 0.00878688,
        1.        ]])

In [196]:
def profile_recommendation2(cast='', director='', genres=''):

        cast = cast.replace(" ", "").replace(',', ' ').lower()
        director = director.replace(" ", "").replace(',', ' ').lower()
        genres = genres.replace(" ", "").replace(',', ' ').lower()

        tags = cast + " " + director + " " + genres
                                 
        
        # cast_vector, director_vector, genres_vector = (cast_tfidf_object.transform([cast]), director_tfidf_object.transform([director]), 
        #                                                genres_tfidf_object.transform([genres]))
        tags_vector = tfidf_tags.transform([tags])


        # cast_similarities, director_similarities, genres_similarities = (cosine_similarity(cast_tfidf_vector, cast_vector), 
        #                                                                 cosine_similarity(director_tfidf_vector, director_vector), 
        #                                                                 cosine_similarity(genres_tfidf_vector, genres_vector))

        tagas_similarities = cosine_similarity(tfidf_tags_matrix, tags_vector)

        #user_input_similarity_scores = (cast_similarities * 2) + (director_similarities) + (genres_similarities * 2)

        user_input_similarity_scores = tagas_similarities

        movies['user_recommendation_score'] = user_input_similarity_scores[:,0]

        recommendations = movies.sort_values(by=['user_recommendation_score'], ascending=False)#[:30]
        
        
        return recommendations['title']#.to_dict(orient='records')

In [204]:
profile_recommendation2(cast='', director='', genres='Science-Fiction')

8530                   Sri Rama Rajyam
0                          29th Street
6341               Munna Bhai M.B.B.S.
6343                                Om
6344                             Oops!
                     ...              
3173       Love in the Time of Cholera
3174             Margot at the Wedding
3175    Mr. Magorium's Wonder Emporium
3176                       August Rush
9519                  Samajavaragamana
Name: title, Length: 9520, dtype: object

In [200]:
profile_recommendation(cast='Robert Downey Jr', director='', genres='Romance, Action, Science-Fiction')

8530               Sri Rama Rajyam
943                    Restoration
3064                     Lucky You
1500           Two Girls and a Guy
702                       Only You
                   ...            
4139                        Pusher
4138    Silent Hill: Revelation 3D
4137                      Fun Size
4136                   Cloud Atlas
4760              God's Not Dead 2
Name: title, Length: 9520, dtype: object

combining all movies

In [121]:
american_movies = pd.read_csv(r'C:\Users\P PAVAN REDDY\Documents\My Files\Projects\Recommendation System - Movies\movies.csv')

In [122]:
american_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6147 entries, 0 to 6146
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   title                         6147 non-null   object
 1   release_year                  6147 non-null   int64 
 2   imdb_id                       6147 non-null   object
 3   poster                        6147 non-null   object
 4   tmdb_id                       6147 non-null   int64 
 5   cast                          6147 non-null   object
 6   director                      6147 non-null   object
 7   genres                        6147 non-null   object
 8   production_companies          6147 non-null   object
 9   cast_display                  6147 non-null   object
 10  director_display              6147 non-null   object
 11  genres_display                6147 non-null   object
 12  production_companies_display  6147 non-null   object
 13  title_search      

In [123]:
american_movies['title_search']

0                         29thstreet
1                    theaddamsfamily
2               alliwantforchristmas
3                  americankickboxer
4       anamericantailfievelgoeswest
                    ...             
6142                        ironman3
6143             avatarthewayofwater
6144         starwarstheforceawakens
6145             starwarsthelastjedi
6146                        furious7
Name: title_search, Length: 6147, dtype: object

In [124]:
american_movies['language'] = 'English'

In [125]:
indian_movies = pickle.load(open(r'C:\Users\P PAVAN REDDY\Documents\My Files\Projects\Recommendation System - Movies\Data\indian_movies.pkl', 'rb'))

In [126]:
indian_movies.head()

,title,release_year,cast_tmdb,director_tmdb,tmdb_id,overview,rating,poster,genres_tmdb,production_companies,imdb_id,language,cast_display,genres_display,director_display,production_companies_display,cast,genres,director
0,Abhay,2001,"[Kamal Haasan, Manisha Koirala, Raveena Tandon...",Suresh Krishna,66346,"Nandu and Vijay are brothers, who were abused ...",7.200,https://image.tmdb.org/t/p/original/5grbkwNnMy...,[Thriller],[V Creations],tt0294264,Hindi,"Kamal Haasan, Manisha Koirala, Raveena Tandon,...",Thriller,Suresh Krishna,V Creations,kamalhaasan manishakoirala raveenatandon golla...,thriller,sureshkrishna
1,Aamdani Atthanni Kharcha Rupaiya,2001,"[Govinda, Juhi Chawla, Johnny Lever, Tabu, Ket...",K Raghavendra Rao,25801,"Jhoomri and her husband, Bhimsha, move into a ...",5.200,https://image.tmdb.org/t/p/original/9xl6TNdI2Q...,[Comedy],no result,tt0305173,Hindi,"Govinda, Juhi Chawla, Johnny Lever, Tabu, Ketk...",Comedy,K Raghavendra Rao,no result,govinda juhichawla johnnylever tabu ketkidave ...,comedy,kraghavendrarao
2,Aashiq,2001,"[Bobby Deol, Karisma Kapoor, Ashok Saraf, Muke...",Indra Kumar,362100,Pooja and Chander Kapoor have a heart-to-heart...,6.000,https://image.tmdb.org/t/p/original/ebH0vv6e5C...,"[Action, Romance]",no result,tt0273406,Hindi,"Bobby Deol, Karisma Kapoor, Ashok Saraf, Mukes...","Action, Romance",Indra Kumar,no result,bobbydeol karismakapoor ashoksaraf mukeshrishi...,action romance,indrakumar
3,Ajnabee,2001,"[Akshay Kumar, Bobby Deol, Kareena Kapoor Khan...",Mustan Alibhai Burmawalla,22316,Raj and Priya befriend their new neighbors Vic...,5.611,https://image.tmdb.org/t/p/original/rIscKIZxR4...,"[Action, Crime, Drama, Thriller]",[Film Folks],tt0278291,Hindi,"Akshay Kumar, Bobby Deol, Kareena Kapoor Khan,...","Action, Crime, Drama, Thriller",Mustan Alibhai Burmawalla,Film Folks,akshaykumar bobbydeol kareenakapoorkhan bipash...,action crime drama thriller,mustanalibhaiburmawalla
4,Aks,2001,"[Amitabh Bachchan, Manoj Bajpayee, Raveena Tan...",Rakeysh Omprakash Mehra,152035,"On the trail of an underworld contract killer,...",5.100,https://image.tmdb.org/t/p/original/4vGi15Vx0K...,"[Crime, Thriller]",no result,tt0289845,Hindi,"Amitabh Bachchan, Manoj Bajpayee, Raveena Tand...","Crime, Thriller",Rakeysh Omprakash Mehra,no result,amitabhbachchan manojbajpayee raveenatandon na...,crime thriller,rakeyshomprakashmehra


In [127]:
clean = lambda s : s.lower().translate({ord(i): '' for i in " :-+;[].*#@$%^&()'\","})

In [128]:
import re

def extract_movie_title(title):
    cleaned_title = re.sub(r'\s*\[[^\]]*\]\s*|\s*\([^)]*\)\s*', '', title)
    return cleaned_title.strip()

In [129]:
extract_movie_title('Adipurush[b]')

'Adipurush'

In [130]:
indian_movies['title']=indian_movies['title'].apply(extract_movie_title)

In [68]:
indian_movies['title_search'] = indian_movies['title'].apply(clean)

In [69]:
indian_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3373 entries, 0 to 3372
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   title                         3373 non-null   object 
 1   release_year                  3373 non-null   int64  
 2   cast_tmdb                     3373 non-null   object 
 3   director_tmdb                 3373 non-null   object 
 4   tmdb_id                       3373 non-null   int64  
 5   overview                      3373 non-null   object 
 6   rating                        3373 non-null   float64
 7   poster                        3373 non-null   object 
 8   genres_tmdb                   3373 non-null   object 
 9   production_companies          3373 non-null   object 
 10  imdb_id                       3373 non-null   object 
 11  language                      3373 non-null   object 
 12  cast_display                  3373 non-null   object 
 13  gen

In [70]:
indian_movies['document'] = (indian_movies['cast'] + " " + indian_movies['director'] + " " 
                       + indian_movies['genres'] + " " + indian_movies['language'])

In [71]:
indian_movies.head()

,title,release_year,cast_tmdb,director_tmdb,tmdb_id,overview,rating,poster,genres_tmdb,production_companies,...,language,cast_display,genres_display,director_display,production_companies_display,cast,genres,director,title_search,document
0,Abhay,2001,"[Kamal Haasan, Manisha Koirala, Raveena Tandon...",Suresh Krishna,66346,"Nandu and Vijay are brothers, who were abused ...",7.200,https://image.tmdb.org/t/p/original/5grbkwNnMy...,[Thriller],[V Creations],...,Hindi,"Kamal Haasan, Manisha Koirala, Raveena Tandon,...",Thriller,Suresh Krishna,V Creations,kamalhaasan manishakoirala raveenatandon golla...,thriller,sureshkrishna,abhay,kamalhaasan manishakoirala raveenatandon golla...
1,Aamdani Atthanni Kharcha Rupaiya,2001,"[Govinda, Juhi Chawla, Johnny Lever, Tabu, Ket...",K Raghavendra Rao,25801,"Jhoomri and her husband, Bhimsha, move into a ...",5.200,https://image.tmdb.org/t/p/original/9xl6TNdI2Q...,[Comedy],no result,...,Hindi,"Govinda, Juhi Chawla, Johnny Lever, Tabu, Ketk...",Comedy,K Raghavendra Rao,no result,govinda juhichawla johnnylever tabu ketkidave ...,comedy,kraghavendrarao,aamdaniatthannikharcharupaiya,govinda juhichawla johnnylever tabu ketkidave ...
2,Aashiq,2001,"[Bobby Deol, Karisma Kapoor, Ashok Saraf, Muke...",Indra Kumar,362100,Pooja and Chander Kapoor have a heart-to-heart...,6.000,https://image.tmdb.org/t/p/original/ebH0vv6e5C...,"[Action, Romance]",no result,...,Hindi,"Bobby Deol, Karisma Kapoor, Ashok Saraf, Mukes...","Action, Romance",Indra Kumar,no result,bobbydeol karismakapoor ashoksaraf mukeshrishi...,action romance,indrakumar,aashiq,bobbydeol karismakapoor ashoksaraf mukeshrishi...
3,Ajnabee,2001,"[Akshay Kumar, Bobby Deol, Kareena Kapoor Khan...",Mustan Alibhai Burmawalla,22316,Raj and Priya befriend their new neighbors Vic...,5.611,https://image.tmdb.org/t/p/original/rIscKIZxR4...,"[Action, Crime, Drama, Thriller]",[Film Folks],...,Hindi,"Akshay Kumar, Bobby Deol, Kareena Kapoor Khan,...","Action, Crime, Drama, Thriller",Mustan Alibhai Burmawalla,Film Folks,akshaykumar bobbydeol kareenakapoorkhan bipash...,action crime drama thriller,mustanalibhaiburmawalla,ajnabee,akshaykumar bobbydeol kareenakapoorkhan bipash...
4,Aks,2001,"[Amitabh Bachchan, Manoj Bajpayee, Raveena Tan...",Rakeysh Omprakash Mehra,152035,"On the trail of an underworld contract killer,...",5.100,https://image.tmdb.org/t/p/original/4vGi15Vx0K...,"[Crime, Thriller]",no result,...,Hindi,"Amitabh Bachchan, Manoj Bajpayee, Raveena Tand...","Crime, Thriller",Rakeysh Omprakash Mehra,no result,amitabhbachchan manojbajpayee raveenatandon na...,crime thriller,rakeyshomprakashmehra,aks,amitabhbachchan manojbajpayee raveenatandon na...


In [72]:
indian_movies = indian_movies[['title', 'release_year', 'imdb_id', 'poster', 'tmdb_id', 'cast',
       'director', 'genres', 'production_companies', 'cast_display',
       'director_display', 'genres_display', 'production_companies_display',
       'title_search', 'document', 'language']]

In [73]:
american_movies.columns

Index(['title', 'release_year', 'imdb_id', 'poster', 'tmdb_id', 'cast',
       'director', 'genres', 'production_companies', 'cast_display',
       'director_display', 'genres_display', 'production_companies_display',
       'title_search', 'document', 'language'],
      dtype='object')

In [74]:
indian_movies.columns

Index(['title', 'release_year', 'imdb_id', 'poster', 'tmdb_id', 'cast',
       'director', 'genres', 'production_companies', 'cast_display',
       'director_display', 'genres_display', 'production_companies_display',
       'title_search', 'document', 'language'],
      dtype='object')

In [75]:
movies_data = pd.concat([american_movies, indian_movies], ignore_index=True)

In [76]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9520 entries, 0 to 9519
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   title                         9520 non-null   object
 1   release_year                  9520 non-null   int64 
 2   imdb_id                       9520 non-null   object
 3   poster                        9520 non-null   object
 4   tmdb_id                       9520 non-null   int64 
 5   cast                          9520 non-null   object
 6   director                      9520 non-null   object
 7   genres                        9520 non-null   object
 8   production_companies          9520 non-null   object
 9   cast_display                  9520 non-null   object
 10  director_display              9520 non-null   object
 11  genres_display                9520 non-null   object
 12  production_companies_display  9520 non-null   object
 13  title_search      

In [77]:
movies_data['title']=movies_data['title'].apply(extract_movie_title)

In [78]:
movies_data['title_search'] = movies_data['title'].apply(clean)

In [79]:
movies_data

,title,release_year,imdb_id,poster,tmdb_id,cast,director,genres,production_companies,cast_display,director_display,genres_display,production_companies_display,title_search,document,language
0,29th Street,1991,tt0101252,https://m.media-amazon.com/images/M/MV5BMWY2MT...,43267,AnthonyLaPaglia DannyAiello LainieKazan FrankP...,GeorgeGallo,Comedy,20thCenturyFox,"Anthony La Paglia, Danny Aiello, Lainie Kaza...",George Gallo,Comedy,20th Century Fox,29thstreet,AnthonyLaPaglia DannyAiello LainieKazan FrankP...,English
1,The Addams Family,1991,tt0101272,https://m.media-amazon.com/images/M/MV5BODc1Nm...,2907,RaúlJuliá AnjelicaHuston ChristopherLloyd Chri...,BarrySonnenfeld,Comedy Fantasy,ScottRudinProductions OrionPictures Paramount,"Raúl Juliá, Anjelica Huston, Christopher Llo...",Barry Sonnenfeld,"Comedy, Fantasy","Scott Rudin Productions, Orion Pictures, Par...",theaddamsfamily,RaúlJuliá AnjelicaHuston ChristopherLloyd Chri...,English
2,All I Want for Christmas,1991,tt0101301,https://m.media-amazon.com/images/M/MV5BMTk3Mj...,13644,HarleyJaneKozak JameySheridan ThoraBirch Ethan...,RobLieberman,Family Drama Comedy,Paramount,"Harley Jane Kozak, Jamey Sheridan, Thora Bir...",Rob Lieberman,"Family, Drama, Comedy",Paramount,alliwantforchristmas,HarleyJaneKozak JameySheridan ThoraBirch Ethan...,English
3,American Kickboxer,1991,tt0101325,https://m.media-amazon.com/images/M/MV5BM2E2Nm...,110830,JohnBarrett KeithVitali TerryNorton BradMorris...,FransNel,Action,GlobalPictures DistantHorizon CannonGroup,"John Barrett, Keith Vitali, Terry Norton, B...",Frans Nel,Action,"Global Pictures, Distant Horizon, Cannon Group",americankickboxer,JohnBarrett KeithVitali TerryNorton BradMorris...,English
4,An American Tail: Fievel Goes West,1991,tt0101329,https://m.media-amazon.com/images/M/MV5BZmUyOT...,10380,PhillipGlasser JamesStewart EricaYohn CathyCav...,SimonWells,Adventure Animation Family Western,AmblinEntertainment UniversalPictures Amblimation,"Phillip Glasser, James Stewart, Erica Yohn, ...",Simon Wells,"Adventure, Animation, Family, Western","Amblin Entertainment, Universal Pictures, Am...",anamericantailfievelgoeswest,PhillipGlasser JamesStewart EricaYohn CathyCav...,English
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9515,Unstoppable,2023,tt22938854,https://image.tmdb.org/t/p/original/mHs0tRWxGC...,1154196,vjsunny saptagiri aksakhan nakshatra posanikri...,diamondratnababu,comedy,[AB2 Productions],"VJ Sunny, Saptagiri, Aksa Khan, Nakshatra, Pos...",Diamond Ratnababu,Comedy,AB2 Productions,unstoppable,vjsunny saptagiri aksakhan nakshatra posanikri...,Telugu
9516,Vimanam,2023,tt26743568,https://image.tmdb.org/t/p/original/d1FQfGMDGR...,1114702,samuthirakani anasuyabharadwaj meerajasmine ma...,sivaprasadyanala,drama family,"[Zee Studios, Kiran Korrapati Creative Works]","Samuthirakani, Anasuya Bharadwaj, Meera Jasmin...",Siva Prasad Yanala,"Drama, Family","Zee Studios, Kiran Korrapati Creative Works",vimanam,samuthirakani anasuyabharadwaj meerajasmine ma...,Telugu
9517,Adipurush,2023,tt12915716,https://image.tmdb.org/t/p/original/kpuOeWclbt...,734253,prabhas saifalikhan kritisanon sunnysinghnijja...,omraut,action adventure drama fantasy history,"[Retrophiles, T-Series, AA Films, UV Creations...","Prabhas, Saif Ali Khan, Kriti Sanon, Sunny Sin...",Om Raut,"Action, Adventure, Drama, Fantasy, History","Retrophiles, TSeries, AA Films, UV Creations, ...",adipurush,prabhas saifalikhan kritisanon sunnysinghnijja...,Telugu
9518,Spy,2023,tt26684398,https://image.tmdb.org/t/p/original/auN2sEpiwc...,1062807,takuyaeguchi atsumitanezaki saorihayami kenich...,takashikatagiri,animation comedy adventure action,"[WIT STUDIO, CloverWorks, TOHO]","Takuya Eguchi, Atsumi Tanezaki, Saori Hayami, ...",Takashi Katagiri,"Animation, Comedy, Adventure, Action","WIT STUDIO, CloverWorks, TOHO",spy,takuyaeguchi atsumitanezaki saorihayami kenich...,Telugu


In [99]:
movies_data.iloc[8779]['poster']

'https://image.tmdb.org/t/p/original/eV90ODRvxea8b2xJS3qnHXqFxqu.jpg'

In [90]:
movies_data['document'] = (movies_data['cast'] + " " + movies_data['director'] + " " 
                       + movies_data['genres'] + " " + movies_data['language'])

In [104]:
movies_data['document'] = movies_data['document'].str.lower()

In [105]:
movies_data['document'][8888]

'rampothineni raashiikhanna sathyaraj raoramesh tulasi naresh santoshsrinivas romance action family telugu'

In [106]:
#movies_data.to_csv('movies_data.csv', index=False)

In [113]:
movies_data[movies_data['release_year'] > 1991].sort_values(by=['release_year'], ascending=False)

,title,release_year,imdb_id,poster,tmdb_id,cast,director,genres,production_companies,cast_display,director_display,genres_display,production_companies_display,title_search,document,language
9519,Samajavaragamana,2023,tt26699538,https://image.tmdb.org/t/p/original/nNE51nnaZi...,1031218,sreevishnu rebamonicajohn naresh srikanthiyeng...,ramabbaraju,comedy drama romance,"[Hasya Movies, AK Entertainments]","Sree Vishnu, Reba Monica John, Naresh, Srikant...",Ram Abbaraju,"Comedy, Drama, Romance","Hasya Movies, AK Entertainments",samajavaragamana,sreevishnu rebamonicajohn naresh srikanthiyeng...,Telugu
7970,Mrs Undercover,2023,tt14339846,https://image.tmdb.org/t/p/original/wEKwOfgo3I...,915865,radhikaapte sumeetvyas rajeshsharma sahebchatt...,anushreemehta,action comedy thriller,[B4U Motion Pictures],"Radhika Apte, Sumeet Vyas, Rajesh Sharma, Sahe...",Anushree Mehta,"Action, Comedy, Thriller",B4U Motion Pictures,mrsundercover,radhikaapte sumeetvyas rajeshsharma sahebchatt...,Hindi
7968,Gaslight,2023,tt27012110,https://image.tmdb.org/t/p/original/8D3lObq5pv...,1092654,saraalikhan vikrantmassey chitrangdasingh rahu...,pavankirpalani,horror thriller,"[Tips Industries, 12th Street Entertainment]","Sara Ali Khan, Vikrant Massey, Chitrangda Sing...",Pavan Kirpalani,"Horror, Thriller","Tips Industries, 12th Street Entertainment",gaslight,saraalikhan vikrantmassey chitrangdasingh rahu...,Hindi
7967,Bholaa,2023,tt15302222,https://image.tmdb.org/t/p/original/xa2CfzV1xK...,965606,ajaydevgn tabu deepakdobriyal amalapaul sanjay...,ajaydevgn,action adventure crime,"[Dream Warrior Pictures, T-Series, Ajay Devgn ...","Ajay Devgn, Tabu, Deepak Dobriyal, Amala Paul,...",Ajay Devgn,"Action, Adventure, Crime","Dream Warrior Pictures, TSeries, Ajay Devgn FF...",bholaa,ajaydevgn tabu deepakdobriyal amalapaul sanjay...,Hindi
7966,Kanjoos Makhichoos,2023,tt15717242,https://image.tmdb.org/t/p/original/sT02AqTJjz...,1099677,kunalkhemu shwetatripathi piyushmishra alkaami...,vipulmehta,comedy drama,"[Soham Rockstar Entertainment, Thundersky Ente...","Kunal Khemu, Shweta Tripathi, Piyush Mishra, A...",Vipul Mehta,"Comedy, Drama","Soham Rockstar Entertainment, Thundersky Enter...",kanjoosmakhichoos,kunalkhemu shwetatripathi piyushmishra alkaami...,Hindi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,Praying with Anger,1992,tt0105162,https://m.media-amazon.com/images/M/MV5BMDM3OD...,114284,MNightShyamalan MikeMuthu RichaAhujaBadami Sus...,MNightShyamalan,Comedy Drama,CrescentMoon,"M Night Shyamalan, Mike Muthu, Richa Ahuja B...",M Night Shyamalan,"Comedy, Drama",Crescent Moon,prayingwithanger,mnightshyamalan mikemuthu richaahujabadami sus...,English
316,The Power of One,1992,tt0105159,https://m.media-amazon.com/images/M/MV5BMjQ1NT...,13823,MorganFreeman StephenDorff SimonFenton GuyWitc...,JohnGAvildsen,Drama,AlcorFilms Canal+ RegencyEnterprises VillageRo...,"Morgan Freeman, Stephen Dorff, Simon Fenton,...",John G Avildsen,Drama,"Alcor Films, Canal+, Regency Enterprises, V...",thepowerofone,morganfreeman stephendorff simonfenton guywitc...,English
315,Poison Ivy,1992,tt0105156,https://m.media-amazon.com/images/M/MV5BZTgzNT...,9264,DrewBarrymore SaraGilbert TomSkerritt CherylLa...,KattShea,Thriller Drama,NewLineCinema,"Drew Barrymore, Sara Gilbert, Tom Skerritt, ...",Katt Shea,"Thriller, Drama",New Line Cinema,poisonivy,drewbarrymore saragilbert tomskerritt cherylla...,English
314,The Player,1992,tt0105151,https://m.media-amazon.com/images/M/MV5BMjM5MD...,10403,TimRobbins GretaScacchi FredWard WhoopiGoldber...,RobertAltman,Comedy Crime Drama Mystery,FineLineFeatures AvenuePictures SpellingEntert...,"Tim Robbins, Greta Scacchi, Fred Ward, Whoo...",Robert Altman,"Comedy, Crime, Drama, Mystery","Fine Line Features, Avenue Pictures, Spellin...",theplayer,timrobbins gretascacchi fredward whoopigoldber...,English
